In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import pickle
from scipy import stats

import flexiznam as flz
from cottage_analysis.analysis import spheres
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, closed_loop_rsof, get_session_list, depth_decoder
from v1_depth_analysis.v1_manuscript_2023 import common_utils as v1_common_utils

## amplitude closedloop vs openloop

In [ ]:
VERSION = 6
SAVE_ROOT = Path(
    f"/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{VERSION}"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)
(SAVE_ROOT/"openloop_separate_recordings").mkdir(parents=True, exist_ok=True)

In [ ]:
import os
import warnings
from pandas.errors import PerformanceWarning
warnings.simplefilter(action='ignore', category=PerformanceWarning)
def extract_amplitude_separate_recordings(neurons_df):
    closedloop_prefix = "rsof_popt_closedloop_g2d_recording_closedloop_"
    openloop_prefix = "rsof_popt_openloop_actual_g2d_recording_openloop_"
    recording_number = np.nanmax([int(i[-1]) for i in neurons_df.columns[neurons_df.columns.str.contains("recording_")]]) + 1
    for i in range(recording_number): 
        neurons_df[f"amplitude_closedloop_recording{i}"] = np.nan
        neurons_df[f"amplitude_openloop_recording{i}"] = np.nan
    neurons_df["recording_order"] = [[np.nan]*recording_number]*len(neurons_df)
    recording_order = np.zeros((recording_number, len(neurons_df)))
    for col in neurons_df.columns:
        if openloop_prefix in col:
            openloop_num = int(col[-1])
            neurons_df[f"amplitude_openloop_recording{openloop_num}"] = neurons_df[col].apply(lambda x: np.exp(x[0]) + x[-1])
            # neurons_df[f"depth_tuning_test_spearmanr_pval_openloop_recording{openloop_num}"] = neurons_df[f"depth_tuning_test_spearmanr_pval_openloop_recording{openloop_num}"]
        if closedloop_prefix in col:
            closedloop_num = int(col[-1])
            neurons_df[f"amplitude_closedloop_recording{closedloop_num}"] = neurons_df[col].apply(lambda x: np.exp(x[0]) + x[-1])
            # neurons_df[f"depth_tuning_test_spearmanr_pval_closedloop_recording{closedloop_num}"] = neurons_df[f"depth_tuning_test_spearmanr_pval_closedloop_recording{closedloop_num}"]
            recording_order[closedloop_num,:] = 1
    neurons_df["recordings_order"] = recording_order.T.tolist()
    return neurons_df


def concatenate_neurons_df_for_separate_recordings(flexilims_session, session_list, cols, filename="neurons_df.pickle", read_iscell=True, verbose=True, recording_number_max=6):
    isess = 0
    for session in session_list:
        cols_new=cols
        neurons_ds = pipeline_utils.create_neurons_ds(
            session_name=session,
            flexilims_session=flexilims_session,
            project=None,
            conflicts="skip",
        )
        if os.path.exists(neurons_ds.path_full.parent / filename):
            neurons_df = pd.read_pickle(neurons_ds.path_full.parent / filename)
            neurons_df = extract_amplitude_separate_recordings(neurons_df)
            if cols is not None:
                cols_new = cols + neurons_df.columns[neurons_df.columns.str.contains("amplitude")].tolist()
                cols_new += ["recordings_order"]   
            if (cols is None) or (set(cols_new).issubset(neurons_df.columns.tolist())):
                if cols is None:
                    neurons_df = neurons_df
                else:
                    if len(neurons_df["recordings_order"].iloc[0]) > 2:
                        neurons_df = neurons_df[cols_new]
                        suite2p_ds = flz.get_datasets(
                            flexilims_session=flexilims_session,
                            origin_name=session,
                            dataset_type="suite2p_rois",
                            filter_datasets={"anatomical_only": 3},
                            allow_multiple=False,
                            return_dataseries=False,
                        )
                        if read_iscell:
                            iscell = np.load(
                                suite2p_ds.path_full / "plane0" / "iscell.npy",
                                allow_pickle=True,
                            )[:, 0]
                            neurons_df["iscell"] = iscell

                        neurons_df["session"] = session
                        
                        if recording_number_max > len(neurons_df["recordings_order"].iloc[0]):
                            for i in range(len(neurons_df["recordings_order"].iloc[0]), recording_number_max): 
                                neurons_df[f"amplitude_closedloop_recording{i}"] = np.nan
                                neurons_df[f"amplitude_openloop_recording{i}"] = np.nan
                        if isess == 0:
                            neurons_df_all = neurons_df
                        else:
                            neurons_df_all = pd.concat(
                                [neurons_df_all, neurons_df], 
                                ignore_index=True,
                            )

                        if verbose:
                            print(f"Finished concat {filename} from session {session}")
                        isess += 1
                    else:
                        print(f"ERROR: SESSION {session}: not more than 2 recordings")
            else:
                print(f"ERROR: SESSION {session}: specified cols not all in neurons_df")
        else:
            print(f"ERROR: SESSION {session}: {filename} not found")
            
    return neurons_df_all

In [ ]:


# project = "hey2_3d-vision_foodres_20220101"
# flexilims_session = flz.get_flexilims_session(project)     
# neurons_df_all = concatenate_neurons_df_for_separate_recordings(flexilims_session=flexilims_session,
#     session_list=[
#     "PZAH8.2h_S20230224",
#     "PZAH8.2h_S20230302",
#     "PZAH8.2h_S20230303",
#     "PZAH8.2h_S20230314",
#     "PZAH8.2h_S20230321",
    
#     "PZAH8.2i_S20230203",
#     "PZAH8.2i_S20230209", 
#     "PZAH8.2i_S20230216",
#     "PZAH8.2i_S20230220", 
#     "PZAH8.2i_S20230324",
#     "PZAH8.2i_S20230330",
#     "PZAH8.2i_S20230404",
    
#     "PZAH8.2f_S20230206",
#     "PZAH8.2f_S20230214",
#     "PZAH8.2f_S20230223",
#     "PZAH8.2f_S20230313",
    
#     "PZAH10.2d_S20230602", 
#     "PZAH10.2d_S20230608", 
#     "PZAH10.2d_S20230613", 
#     "PZAH10.2d_S20230623", 
#     "PZAH10.2d_S20230818", 
#     "PZAH10.2d_S20230821", 
#     "PZAH10.2d_S20230920",
#     "PZAH10.2d_S20230922", 
    
#     "PZAH10.2f_S20230606", 
#     "PZAH10.2f_S20230609", 
#     "PZAH10.2f_S20230615", 
#     "PZAH10.2f_S20230623",
#     "PZAH10.2f_S20230627",
#     "PZAH10.2f_S20230822", 
#     "PZAH10.2f_S20230908", 
#     "PZAH10.2f_S20230924",
#     ], 
#                                                cols=[
#                                                     "roi",
#                                                     "best_depth",
#                                                     "preferred_depth_closedloop",
#                                                     "preferred_depth_closedloop_crossval",
#                                                     "depth_tuning_test_rsq_closedloop",
#                                                     "depth_tuning_test_spearmanr_pval_closedloop",
#                                                     "depth_tuning_test_spearmanr_rval_closedloop",
#                                                     "preferred_RS_closedloop_g2d",
#                                                     "preferred_RS_closedloop_crossval_g2d",
#                                                     "preferred_OF_closedloop_g2d",
#                                                     "preferred_OF_closedloop_crossval_g2d",
#                                                     "rsof_rsq_closedloop_crossval_g2d",
#                                                     "preferred_RS_openloop_actual_g2d",
#                                                     "preferred_OF_openloop_actual_g2d",
#                                                     "rsof_popt_closedloop_g2d",
#                                                     "rsof_popt_openloop_actual_g2d",
#                                                     "rsof_rsq_closedloop_g2d",
#                                                     "rsof_rsq_openloop_actual_g2d",
#                                                 ], 
#                                                filename="neurons_df.pickle", 
#                                                read_iscell=True, 
#                                                verbose=True)
# neurons_df_all.to_pickle(SAVE_ROOT / "openloop_separate_recordings" / "neurons_df_multiple_recordings_only.pickle")

In [ ]:
neurons_df_all = pd.read_pickle(SAVE_ROOT / "openloop_separate_recordings" / "neurons_df_multiple_recordings_only.pickle")
neurons_df_amp = neurons_df_all[(neurons_df_all["iscell"] ==1)
                                & (neurons_df_all["amplitude_closedloop_recording0"] > 0.5)
]
i = 2
j = 1
if i < j:
        sfx = "before"
elif i > j:
        sfx = "after"
ratios = neurons_df_amp[f"amplitude_closedloop_recording{i}"] / neurons_df_amp[f"amplitude_openloop_recording{j}"]
print(f"closed loop amp / open loop amp median: {np.nanmedian(ratios)}")
print(f"1-sample wilcoxon test: {stats.wilcoxon(ratios-1)}")
bins = np.geomspace(1e-1, 1e1, 21)
fontsize_dict = {"title": 8, "label": 7, "tick": 5, "legend": 5}
fig = plt.figure(figsize=(1.5,1.5), dpi=300)
ax = fig.add_subplot(111)
ax.hist(ratios, bins=bins, 
        color="cornflowerblue",
        edgecolor="royalblue",
        alpha=1)
ax.set_xscale("log")
ax.tick_params(axis='both', which='major', labelsize=fontsize_dict["tick"])
ax.set_title(f"closed loop {sfx}", fontsize=fontsize_dict["title"])
ax.set_xlabel(f"Peak response ratio of\nclosed loop to open loop", fontsize=fontsize_dict["label"])
ax.set_ylabel("Number of neurons", fontsize=fontsize_dict["label"])
ax.axvline(1, color="k", linestyle="dotted", linewidth=1)
sns.despine()

## decoder


In [ ]:
def concatenate_decoder_df_for_separate_recordings(flexilims_session, session_list,):
    decoder_df_all = pd.DataFrame(columns=[[
        "session",
        "accuracy_closedloop0",
        "accuracy_openloop1",
        "accuracy_closedloop2",
        "conmat_closedloop0",
        "conmat_openloop1",
        "conmat_closedloop2",
        ]
        ],
        index = range(len(session_list))
    )
    for isess, session in enumerate(session_list):
        print(session)
        neurons_ds = pipeline_utils.create_neurons_ds(
            session_name=session,
            flexilims_session=flexilims_session,
            project=None,
            conflicts="skip",
        )
        neurons_df = pd.read_pickle(neurons_ds.path_full)
        neurons_df = extract_amplitude_separate_recordings(neurons_df)

        if len(neurons_df["recordings_order"].iloc[0]) > 2:
            decoder_df_all.loc[isess, "session"] = session
            for i, rec in enumerate(neurons_df["recordings_order"].iloc[0][:3]):
                if rec == 1:
                    sfx = "closedloop"
                else:
                    sfx = "openloop"
                if os.path.exists(neurons_ds.path_full.parent / f"decoder_results_{sfx}{i}.pickle"):
                    decoder_df = pd.read_pickle(neurons_ds.path_full.parent / f"decoder_results_{sfx}{i}.pickle")
                    if f"accuracy_{sfx}" not in decoder_df.keys():
                        print(f"ERROR: SESSION {session}: {sfx}{i} decoder_results not found")
                    else:
                        decoder_df_all.loc[isess, f"accuracy_{sfx}{i}"] = decoder_df[f"accuracy_{sfx}"]
                        decoder_df_all.loc[isess, f"conmat_{sfx}{i}"] = decoder_df[f"conmat_{sfx}"].reshape(1, decoder_df[f"conmat_{sfx}"].shape[0], decoder_df[f"conmat_{sfx}"].shape[1])
                else:
                    print(f"ERROR: SESSION {session}: {sfx}{i} decoder_results not found")
        else:
            print(f"ERROR: SESSION {session}: not more than 2 recordings")
            
    return decoder_df_all

In [ ]:
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)    
decoder_df_all = concatenate_decoder_df_for_separate_recordings(flexilims_session=flexilims_session,
                                                                session_list =  [
                                                                    "PZAH8.2h_S20230224",
                                                                    "PZAH8.2h_S20230303",
                                                                    "PZAH8.2h_S20230314",
                                                                    
                                                                    "PZAH8.2i_S20230203",
                                                                    "PZAH8.2i_S20230209",
                                                                    "PZAH8.2i_S20230216",
                                                                    
                                                                    "PZAH8.2f_S20230214",
                                                                    "PZAH8.2f_S20230313",
 
                                                                    "PZAH10.2f_S20230615", 
                                                                    "PZAH10.2f_S20230822",
                                                                    "PZAH10.2f_S20230908",
                                                                ],)
                                                            

In [ ]:
select = decoder_df_all.accuracy_closedloop2.notnull().values
for i, col in enumerate(["accuracy_closedloop0", "accuracy_openloop1", "accuracy_closedloop2"]):
    y = decoder_df_all[select][col].dropna().values.flatten()
    sns.stripplot(
        x=np.ones(len(y)) * i,
        y=y,
        size=10,
        alpha=1,
        jitter=0.1,
        edgecolor="white",
        color=sns.color_palette("Set1")[i],
    )
    plt.plot(
        [i - 0.2, i + 0.2],
        [np.nanmean(y), np.nanmean(y)],
        linewidth=3,
        color="k"
    )
plt.xticks([0, 1, 2], ["Closed loop before", "Open loop", "Closed loop after"])
plt.ylabel("Decoder accuracy")
sns.despine()



In [ ]:
select = ((decoder_df_all.accuracy_closedloop2.notnull().values)
          & (decoder_df_all.accuracy_closedloop0.notnull().values)
).flatten()
plt.figure(figsize=(5, 5))
plt.plot(
    decoder_df_all[select][["accuracy_closedloop0",
                            "accuracy_openloop1",
                            "accuracy_closedloop2"]].values.T,
    'o-',
    alpha=0.5
)
plt.xticks([0, 1, 2], ["Closed loop before", "Open loop", "Closed loop after"])
plt.ylabel("Decoder accuracy")
sns.despine()